In [7]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [8]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'movie-ml-latest-small/merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'movie-ml-latest-small/title_large_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'movie-ml-latest-small/title_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/title_large_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/title_large_1_test_predictions_few_shot.csv


# Data Overview

In [9]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3405 entries, 0 to 3404
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3405 non-null   int64  
 1   imdbId   3405 non-null   int64  
 2   tmdbId   3405 non-null   float64
 3   title    3405 non-null   object 
 4   genres   3405 non-null   object 
 5   userId   3405 non-null   int64  
 6   rating   3405 non-null   float64
 7   tag      3405 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 212.9+ KB


,movieId,imdbId,tmdbId,title,genres,userId,rating,tag
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar
1,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,pixar
2,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun
3,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy
4,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game


In [10]:
MOVIELENS_CONTENT_SYSTEM = "MovieLens movies critic"

@retry_decorator
def predict_rating_combined_ChatCompletion(combined_text, 
                                           model=GPT_MODEL_NAME, 
                                           temperature=TEMPERATURE, 
                                           approach="zero-shot", 
                                           rating_history=None, 
                                           similar_users_ratings=None, 
                                           seed=RANDOM_STATE, 
                                           system_content=MOVIELENS_CONTENT_SYSTEM):
    """
    Predicts product ratings using different approaches with the GPT model.
    """
    # Validation
    if approach == "few-shot" and rating_history is None:
        raise ValueError("Rating history is required for the few-shot approach.")
    if approach == "CF" and similar_users_ratings is None:
        raise ValueError("Similar users' ratings are required for the collaborative filtering approach.")
    if not system_content:
        raise ValueError("System content is required.")

    # Check and reduce length of combined_text
    combined_text = check_and_reduce_length(combined_text, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
    prompt = f"How will user rate this {combined_text}? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text."

    # Construct the prompt based on the approach
    if approach == "few-shot":
        rating_history = check_and_reduce_length(rating_history, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
        prompt += f"\n\nHere is user rating history:\n{rating_history}"

    elif approach == "CF":
        similar_users_ratings = check_and_reduce_length(similar_users_ratings, MAX_TOKENS_CHAT_GPT // 3, TOKENIZER)
        prompt += f"\n\nHere are the rating history from users who are similar to this user:\n{similar_users_ratings}"

    # Adding end of the prompt
    prompt += "\n\nBased on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)"

    print(f"Constructed Prompt for {approach} approach:\n")
    # meaningful print for prompt
    print(f'The prompt:\n**********\n{prompt}\n**********\n')

    try:
        # Create the API call
        response = openai.ChatCompletion.create(
            model=model,
            temperature=temperature,
            max_tokens=MAX_TOKENS_CHAT_GPT,
            seed=seed,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": prompt}
            ]
        )
    except Exception as e:
        print(f"Error occurred during API call: {e}")
        return None, str(e)  # Include the error message with the response

    # Extract the system fingerprint and print it
    system_fingerprint = response.get('system_fingerprint')
    print(f"\n\nSystem Fingerprint: {system_fingerprint}")

    # Extract and return the rating
    rating_text = response.choices[0].message['content'].strip()
    print(f'\nAPI call response: "{rating_text}"')
    extracted_rating = extract_numeric_rating(rating_text)
    return extracted_rating

# Zero-shot (OpenAI API)

In [11]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=['title'],
                                       user_column_name='userId',
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this title: Toy Story (1995)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



System Fingerprint: fp_eeff13170a

API call response: "5"
Processing item 1/3405

Details: Toy Story (1995)

Predicted Rating: 5.0 stars

------------------------------------

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this title: Toy Story (1995)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output shou

In [ ]:
def evaluate_model_predictions_rmse_mae(data_path, num_examples, actual_ratings_column, predicted_ratings_column):
    """
    Evaluate model predictions with RMSE and MAE, and display a few prediction results.

    Parameters:
    - data_path (str): Path to the CSV file containing the model's predictions.
    - num_examples (int): Number of examples to show for debugging purposes.
    - actual_ratings_column (str): Name of the column in the CSV file containing actual ratings.
    - predicted_ratings_column (str): Name of the column in the CSV file containing predicted ratings.
    """
    # Read the data from the CSV file
    data = pd.read_csv(data_path)

    # Process predicted ratings to extract valid numerical values
    data[predicted_ratings_column] = data[predicted_ratings_column].apply(lambda x: x if isinstance(x, float) else None)

    # Extract the actual and predicted ratings
    actual_ratings = data[actual_ratings_column].tolist()
    predicted_ratings = data[predicted_ratings_column].tolist()

    # Filter out invalid (None) predictions
    filtered_ratings = [(actual, predicted) for actual, predicted in zip(actual_ratings, predicted_ratings) if predicted is not None]
    
    # Check if there are valid predictions for evaluation
    if not filtered_ratings:
        print("No valid predictions available for evaluation.")
        return

    # Unpack the filtered actual and predicted ratings
    actual_filtered, predicted_filtered = zip(*filtered_ratings)

    # Calculate RMSE and MAE using the custom function
    rmse, mae = calculate_rmse_and_mae(actual_filtered, predicted_filtered)

    # Output the evaluation results
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')

    # Display the first few actual vs. predicted ratings for debugging
    print("\nFirst few actual vs predicted ratings:")
    for actual, predicted in actual_vs_predicted[:num_examples]:
        print(f"Actual: {actual}, Predicted: {predicted}")

evaluate_model_predictions_rmse_mae(
    data_path=ZERO_SHOT_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='rating',
    predicted_ratings_column='zero_shot_predicted_rating'
)


# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [ ]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['title'],
                                       columns_for_prediction=['title'],
                                       user_column_name='userId',
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


In [ ]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='few_shot_predicted_rating'
)


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication